In [1]:
from datasets import load_dataset
import os, sys
import numpy as np
import torch, pandas, torch.nn, tempfile,os,pickle
from collections import namedtuple
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm
from accelerate import Accelerator
from torch import nn
from peft import get_peft_config, get_peft_model, LoraConfig, PrefixTuningConfig, PromptTuningInit, PromptTuningConfig, TaskType, PeftType
from transformers import BitsAndBytesConfig,LlamaTokenizer,LlamaForCausalLM,GPT2Tokenizer, GPT2LMHeadModel
import torch

from peft import prepare_model_for_kbit_training
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
device = 'cuda'
model_path ="meta-llama/Llama-2-7b-chat-hf"
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)
model =LlamaForCausalLM.from_pretrained(model_path,
                                    #load_in_8bit=True, #  7.7GB로
                                        quantization_config =nf4_config, #  4.4GB로 
                                        device_map="auto", # gpu 꽉차면 cpu로 올려줌 
                                        
                                       )
tokenizer = LlamaTokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token

/home/wooseok/anaconda3/envs/peft/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████| 2/2 [00:03<00:00,  1.76s/it]


In [2]:
with open('/home/wooseok/peft/FT_input_datasets_llama.pickle', 'rb') as fr:
    data = pickle.load(fr)
del_index = []
for i in range(len(data)):
    if len(data['context'][i]) %2 ==0:
        del_index.append(i)
data.drop(del_index, axis=0, inplace=True)
data=data.reset_index(drop = True)
data = data[:10]
data     
        

,context,reference,act,act_label,input_tokens
0,"[Hey man , you wanna buy some weed ? ]",Some what ?,2,Question,"[1, 894, 2, 1, 18637, 767, 1919, 366, 281, 971..."
1,"[Hey man , you wanna buy some weed ? , Some w...","Oh , umm , no thanks .",4,Commissive,"[1, 1876, 790, 573, 2, 1, 18637, 767, 1919, 36..."
2,[The taxi drivers are on strike again . ],What for ?,2,Question,"[1, 894, 2, 1, 450, 8818, 29875, 18563, 526, 3..."
3,"[The taxi drivers are on strike again . , Wha...",It is really a hot potato .,1,Inform,"[1, 15162, 2, 1, 450, 8818, 29875, 18563, 526,..."
4,[We've managed to reduce our energy consumptio...,That's excellent . How have you managed that ?,2,Question,"[1, 894, 2, 1, 1334, 29915, 345, 8745, 304, 10..."
5,[We've managed to reduce our energy consumptio...,What does that mean exactly ?,2,Question,"[1, 894, 2, 1, 1334, 29915, 345, 8745, 304, 10..."
6,"[Believe it or not , tea is the most popular b...","Well , people from Asia to Europe all enjoy t...",1,Inform,"[1, 15162, 2, 1, 3741, 2418, 372, 470, 451, 19..."
7,"[Believe it or not , tea is the most popular b...","Yes , Chinese people love drinking tea so muc...",1,Inform,"[1, 15162, 2, 1, 3741, 2418, 372, 470, 451, 19..."
8,[What are your personal weaknesses ? ],I ’ m afraid I ’ m a poor talker . I ’ m not ...,1,Inform,"[1, 15162, 2, 1, 1724, 526, 596, 7333, 8062, 2..."
9,"[What are your personal weaknesses ? , I ’ m ...",I don ’ t try to lead people . I ’ d rather c...,1,Inform,"[1, 15162, 2, 1, 1724, 526, 596, 7333, 8062, 2..."


In [3]:
def generate_sentence(prompt,j):

    print(prompt)    
    print(" *******************    ")
    
    input_id = tokenizer.encode(prompt,return_tensors ='pt')
    output_so_far  = input_id
    last = output_so_far[:, -1:] # 마지막 단어
    past = model(output_so_far[:, :-1],return_dict =True)["past_key_values"] #이전단어들의 key_value
    
    
    for i in range(10):
        lm_output= model(last, past_key_values=past,use_cache = True,
                         return_dict=True,output_hidden_states=True)  
        logit, past = (
            lm_output["logits"], 
            # torch.Size([1, 1, 32000]
            
            lm_output["past_key_values"]) 
            # 32, 2, torch.Size([1, 32, Sequence_len, 128]) )
        probs = nn.functional.softmax(logit[:,-1,:], dim=-1)
        last = torch.topk(probs, k=1, dim=-1)[1]
        
        output_so_far = torch.cat((output_so_far,last),dim=1)
        
        del lm_output
        
        torch.cuda.empty_cache()
        if last[0] ==torch.tensor(tokenizer.eos_token_id):
            break
    print(tokenizer.decode(output_so_far[0][len(input_id[0]):])) 
    return tokenizer.decode(output_so_far[0][len(input_id[0]):])

gen = []
for j in tqdm(range(len(data))):
    gen.append(generate_sentence(data['prompt'][j],j))
data['gen'] = gen
data

  0%|                                                                                            | 0/10 [00:00<?, ?it/s]


KeyError: 'prompt'